In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm


test = pd.read_csv('round1_ijcai_18_test_b_20180418.txt', sep=' ')

train = pd.read_csv('round1_ijcai_18_train_20180301 (1).zip', sep=' ')

sample_sub = pd.read_csv('round1_ijcai_18_result_demo_20180301.txt', sep=' ')

train.nunique()

import time
def preprocess(train):
    def timestamp2datetime(ts):
        dt = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ts))
        return dt
    train['Timestamp'] = train.context_timestamp.apply(timestamp2datetime)
    train = train.drop(['context_timestamp'], axis=1)
    def time_preprocess(X):
        X['Timestamp'] = pd.to_datetime(X['Timestamp'])
#         X['year'] = X['Timestamp'].dt.year
#         X['month'] = X['Timestamp'].dt.month 
        X['day'] = X['Timestamp'].dt.day
        X['week_day'] = X['Timestamp'].dt.weekday
        X['hour'] = X['Timestamp'].dt.hour
        X['minute'] = X['Timestamp'].dt.minute
        X['minute'] = X['minute'] // 15 * 15
        X = X.drop('Timestamp', axis=1)    
        return X
    train = time_preprocess(train)

    train['item_category_list'] = pd.to_numeric(train.item_category_list.str.split(';', expand=True)[1])

    df = train.item_property_list.str.split(';', expand=True)

    df = df.iloc[:, :21].fillna(-1).apply(pd.to_numeric)
    train = pd.concat([train, df], axis=1).reset_index(drop=True).drop('item_property_list', axis=1)

    train['predict_category_property'] = train.predict_category_property.str.split(';',
                                                                                   expand=True).isnull().sum(axis=1)
    train.columns = list(map(str, train.columns.tolist()))

    return train
train = preprocess(train)
test = preprocess(test)

In [8]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train)
rejected_variables = profile.get_rejected_variables(threshold=0.9)
rejected_variables

['shop_score_delivery',
 'shop_score_description',
 'shop_star_level',
 'user_occupation_id',
 'user_star_level']

In [2]:
# 'user_id' - not sure
fields = ['context_page_id', 'item_brand_id', 'shop_id']
dict_field = dict(zip(fields, range(len(fields))))
dict_field

concated = pd.concat([test, train.drop('is_trade', axis=1)], ignore_index=True)#.reset_index(drop=True)


keys = []
for fieldname in dict_field:
    keys += [fieldname + '_' + str(i) for i in concated[fieldname].unique()]

In [3]:
fields = ['context_page_id', 'item_brand_id', 'shop_id']
concated = pd.concat([test, train.drop('is_trade', axis=1)], ignore_index=True).reset_index(drop=True)

In [42]:
# def format_data(concated, fields):
dict_field = dict(zip(fields, range(len(fields))))
keys = []
for fieldname in dict_field:
    keys += [fieldname + '_' + str(i) for i in concated[fieldname].unique()]
dict_features = dict(zip(keys, range(len(keys))))
('context_page_id', '405', 1)


In [15]:
cols4ftrl = train.nunique()[(train.nunique() <= 2055) & (train.nunique() > 2) ].index.tolist()

In [16]:
n_val = 380000

In [17]:
dumm_df = pd.get_dummies(concated[cols4ftrl], columns=cols4ftrl, sparse=True) #.to_coo().tocsr()


In [ ]:
X = dumm_df.iloc[:train.shape[0],:]
y = train.is_trade.values

In [18]:
X = dumm_df.iloc[:n_val,:].to_coo().tocsr()
y = train.is_trade[:n_val].values

In [20]:
X_val = dumm_df.iloc[n_val:train.shape[0],:].to_coo().tocsr()
y_val = train.is_trade[n_val:].values

In [ ]:
import numpy as np
import scipy.sparse as sp
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

import ftrl

# X = [
#     [1, 1, 0, 0, 0],
#     [1, 1, 1, 0, 0],
#     [0, 1, 1, 1, 0],
#     [0, 0, 1, 1, 1],
#     [0, 0, 0, 1, 1],   
# ]

# X = sp.csr_matrix(X)
# y = np.array([1, 1, 1, 0, 0], dtype='float32')

model = ftrl.FtrlProximal(alpha=0.1, beta=0.1, l1=1, l2=10)

# make 10 passes over the data
for i in range(1000):
    model.fit(X, y)
    y_pred = model.predict(X_val)
    auc = roc_auc_score(y_val, y_pred)
    print('%02d: %.5f' % (i + 1, auc))

1: 0.49188000656101627, log:<function log_loss at 0x7f3bbebeb9d8>
2: 0.5029212704921837, log:<function log_loss at 0x7f3bbebeb9d8>
3: 0.5031150720661436, log:<function log_loss at 0x7f3bbebeb9d8>
4: 0.5002815820354197, log:<function log_loss at 0x7f3bbebeb9d8>
5: 0.5018038642089482, log:<function log_loss at 0x7f3bbebeb9d8>
6: 0.5008453476141692, log:<function log_loss at 0x7f3bbebeb9d8>
7: 0.5003097582536656, log:<function log_loss at 0x7f3bbebeb9d8>
8: 0.5079092946834273, log:<function log_loss at 0x7f3bbebeb9d8>
9: 0.4935127249761717, log:<function log_loss at 0x7f3bbebeb9d8>
10: 0.498137529989902, log:<function log_loss at 0x7f3bbebeb9d8>
11: 0.4996871426065946, log:<function log_loss at 0x7f3bbebeb9d8>
12: 0.5023426901094756, log:<function log_loss at 0x7f3bbebeb9d8>
13: 0.4914665263596033, log:<function log_loss at 0x7f3bbebeb9d8>
14: 0.5012195374410278, log:<function log_loss at 0x7f3bbebeb9d8>
15: 0.49438700603681884, log:<function log_loss at 0x7f3bbebeb9d8>
16: 0.508572272427

/home/magas/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


19: 0.49979797883571514, log:<function log_loss at 0x7f3bbebeb9d8>
20: 0.5012624704497789, log:<function log_loss at 0x7f3bbebeb9d8>
21: 0.49754568284643563, log:<function log_loss at 0x7f3bbebeb9d8>
22: 0.5010062586134711, log:<function log_loss at 0x7f3bbebeb9d8>
23: 0.49603647812660756, log:<function log_loss at 0x7f3bbebeb9d8>
24: 0.49664513085148115, log:<function log_loss at 0x7f3bbebeb9d8>
25: 0.500280873660622, log:<function log_loss at 0x7f3bbebeb9d8>
26: 0.4985953874297182, log:<function log_loss at 0x7f3bbebeb9d8>
27: 0.500943231576515, log:<function log_loss at 0x7f3bbebeb9d8>
28: 0.5045957677537045, log:<function log_loss at 0x7f3bbebeb9d8>
29: 0.496326737753299, log:<function log_loss at 0x7f3bbebeb9d8>
30: 0.5012531516571818, log:<function log_loss at 0x7f3bbebeb9d8>
31: 0.5064627933465531, log:<function log_loss at 0x7f3bbebeb9d8>
32: 0.5045065064225167, log:<function log_loss at 0x7f3bbebeb9d8>
33: 0.5033653543168605, log:<function log_loss at 0x7f3bbebeb9d8>
34: 0.500

/home/magas/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


53: 0.500051952574063, log:<function log_loss at 0x7f3bbebeb9d8>
54: 0.5034932923013339, log:<function log_loss at 0x7f3bbebeb9d8>
55: 0.5027909631161456, log:<function log_loss at 0x7f3bbebeb9d8>
56: 0.5061851348525774, log:<function log_loss at 0x7f3bbebeb9d8>
57: 0.5087059690103124, log:<function log_loss at 0x7f3bbebeb9d8>
58: 0.5020824722915533, log:<function log_loss at 0x7f3bbebeb9d8>
59: 0.5029156096004815, log:<function log_loss at 0x7f3bbebeb9d8>
60: 0.5043772616086751, log:<function log_loss at 0x7f3bbebeb9d8>
61: 0.5047974744240058, log:<function log_loss at 0x7f3bbebeb9d8>
62: 0.5037517941423755, log:<function log_loss at 0x7f3bbebeb9d8>
63: 0.5006450424267651, log:<function log_loss at 0x7f3bbebeb9d8>
64: 0.5030710154284032, log:<function log_loss at 0x7f3bbebeb9d8>
65: 0.5037523590102099, log:<function log_loss at 0x7f3bbebeb9d8>
66: 0.5036774514286942, log:<function log_loss at 0x7f3bbebeb9d8>
67: 0.5058970675237513, log:<function log_loss at 0x7f3bbebeb9d8>
68: 0.50195

177: 0.5050989123271382, log:<function log_loss at 0x7f3bbebeb9d8>
178: 0.5060588914716071, log:<function log_loss at 0x7f3bbebeb9d8>
179: 0.5053367308454233, log:<function log_loss at 0x7f3bbebeb9d8>
180: 0.5084562913208454, log:<function log_loss at 0x7f3bbebeb9d8>
181: 0.5052316348948356, log:<function log_loss at 0x7f3bbebeb9d8>
182: 0.5052025060346205, log:<function log_loss at 0x7f3bbebeb9d8>
183: 0.5047176143255856, log:<function log_loss at 0x7f3bbebeb9d8>
184: 0.5057189814889452, log:<function log_loss at 0x7f3bbebeb9d8>
185: 0.5067882671393724, log:<function log_loss at 0x7f3bbebeb9d8>
186: 0.5044986684996475, log:<function log_loss at 0x7f3bbebeb9d8>
187: 0.5068438379209151, log:<function log_loss at 0x7f3bbebeb9d8>
188: 0.5014556308224134, log:<function log_loss at 0x7f3bbebeb9d8>
189: 0.505786057254271, log:<function log_loss at 0x7f3bbebeb9d8>
190: 0.5050908209770785, log:<function log_loss at 0x7f3bbebeb9d8>
191: 0.5012420771942886, log:<function log_loss at 0x7f3bbebeb9

301: 0.5059251948885628, log:<function log_loss at 0x7f3bbebeb9d8>
302: 0.5043857956929835, log:<function log_loss at 0x7f3bbebeb9d8>
303: 0.5049514757156905, log:<function log_loss at 0x7f3bbebeb9d8>
304: 0.5052707756557474, log:<function log_loss at 0x7f3bbebeb9d8>
305: 0.5013232105353408, log:<function log_loss at 0x7f3bbebeb9d8>
306: 0.5031597576918511, log:<function log_loss at 0x7f3bbebeb9d8>
307: 0.5047512224350603, log:<function log_loss at 0x7f3bbebeb9d8>
308: 0.5044861803404986, log:<function log_loss at 0x7f3bbebeb9d8>
309: 0.5063388705037131, log:<function log_loss at 0x7f3bbebeb9d8>
310: 0.5030243542919451, log:<function log_loss at 0x7f3bbebeb9d8>
311: 0.5042190192282232, log:<function log_loss at 0x7f3bbebeb9d8>
312: 0.5047655700780531, log:<function log_loss at 0x7f3bbebeb9d8>
313: 0.5021892994857199, log:<function log_loss at 0x7f3bbebeb9d8>
314: 0.5023620788162226, log:<function log_loss at 0x7f3bbebeb9d8>
315: 0.5039573571806488, log:<function log_loss at 0x7f3bbebeb

In [29]:
train

,instance_id,item_id,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,11,12,13,14,15,16,17,18,19,20
0,108641074714126964,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,8838717711606756251,9172976955054793469,6296485275518770064,8154150695030794787,8199832121759092112,773321667872991333,7270669313837600482,3408398779125901630,4359277201620147874,448184584262668803
1,5754713551599725161,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,8838717711606756251,9172976955054793469,6296485275518770064,8154150695030794787,8199832121759092112,773321667872991333,7270669313837600482,3408398779125901630,4359277201620147874,448184584262668803
2,842679481291040981,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,8838717711606756251,9172976955054793469,6296485275518770064,8154150695030794787,8199832121759092112,773321667872991333,7270669313837600482,3408398779125901630,4359277201620147874,448184584262668803
3,937088850059189027,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2681414445369714628,...,8838717711606756251,9172976955054793469,6296485275518770064,8154150695030794787,8199832121759092112,773321667872991333,7270669313837600482,3408398779125901630,4359277201620147874,448184584262668803
4,7975697065017708072,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2729475788342039013,...,8838717711606756251,9172976955054793469,6296485275518770064,8154150695030794787,8199832121759092112,773321667872991333,7270669313837600482,3408398779125901630,4359277201620147874,448184584262668803
5,7764762765372067286,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,4512655448325954611,...,8838717711606756251,9172976955054793469,6296485275518770064,8154150695030794787,8199832121759092112,773321667872991333,7270669313837600482,3408398779125901630,4359277201620147874,448184584262668803
6,6956333474094867789,285660928590172217,8277336076276184272,9057103201734987852,548352491538518780,8,9,8,13,8811056487516803043,...,1134982063610307090,71803110314516845,1301076623647253687,3258780649701680217,514980552440578167,6411614163944830538,478060273908663971,1418324867218214039,6457208937217973355,4859870894564764703
7,8387099821892927911,285660928590172217,8277336076276184272,9057103201734987852,548352491538518780,8,9,8,13,6507704883896466138,...,1134982063610307090,71803110314516845,1301076623647253687,3258780649701680217,514980552440578167,6411614163944830538,478060273908663971,1418324867218214039,6457208937217973355,4859870894564764703
8,4021878205550012615,5202355029344881809,5755694407684602296,5520678735822176314,548352491538518780,8,9,10,16,6203308008480593423,...,7271630337430743106,8247723580205204722,4564647423883133001,4224097012885011005,542901723744518864,4444792543859265070,1665611025031010859,1266232729783845119,7860359346697893790,549707957212148300
9,6499571365974135517,285660928590172217,8277336076276184272,9057103201734987852,548352491538518780,8,9,8,13,6041712044514783312,...,1134982063610307090,71803110314516845,1301076623647253687,3258780649701680217,514980552440578167,6411614163944830538,478060273908663971,1418324867218214039,6457208937217973355,4859870894564764703
